In [7]:
import json
import requests
import numpy as np
import pandas as pd 
from tqdm import tqdm
from time import sleep
from dotenv import dotenv_values


import json
import os
from geopy.geocoders import Nominatim

In [ ]:
pd.set_option('display.max_columns', None)

In [8]:
def get_vacancies_id(header:dict, param:dict=None, URL:str='https://api.hh.ru/vacancies', per_page:int=100, time_delay:float=0)->json:
    sleep(time_delay)
    return requests.get(url=URL, headers=header, params=param).json()

In [9]:
JOB_TITLE = 'Аналитик данных'

USER_AGENT = dotenv_values('.env')['USER_AGENT']
CACHE_FILE = dotenv_values('.env')['CACHE_FILE']


BASE_URL='https://api.hh.ru/vacancies'

PER_PAGE = 100

header = {'User-Agent':USER_AGENT}
param = {'text': JOB_TITLE,
          'search_field': 'name',
          'page': 0,
          'per_page': PER_PAGE,
          'only_with_salary': True,
          'locale': 'RU'}

In [10]:
resp = get_vacancies_id(header, param, per_page=PER_PAGE)

VAC_CNT = resp['found']
PAGES_CNT = resp['pages']
current_page = resp['page']
VAC_CNT, PAGES_CNT, current_page

(209, 3, 0)

In [11]:
vac_ids = []
current_page = 0
PAGES_CNT = 999

while current_page <= PAGES_CNT-1:
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']
    PAGES_CNT = resp['pages']

    for item in resp['items']:
        vac_ids.append(item['id'])

    param['page'] = current_page + 1
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']


print("It's all OK" if len(set(vac_ids)) == VAC_CNT else f"Smt went WRONG\n{len(set(vac_ids))}___{VAC_CNT}")

Smt went WRONG
210___209


In [12]:
def get_skills_str(key_skills:list)->str:
    '''Функция преобразования списка словарей требуемых скилов в строку'''
    result = []
    for skill in key_skills:
        result.append(skill.get('name'))
    return '; '.join(result)

In [55]:
data_list = []

for vac_id in tqdm(vac_ids, desc='Получение описания вакансий...'):
    try:
        vac_annote = get_vacancies_id(header=header, URL=BASE_URL+f'/{vac_id}', time_delay=0.4)


        address = vac_annote.get('address')
        employer = vac_annote.get('employer')
        salary = vac_annote.get('salary')
        salary_range = vac_annote.get('salary_range', {})
        mode = salary_range.get('mode', {}) if salary_range else {}
        prof_roles = vac_annote.get('professional_roles', [])
        
        name = vac_annote.get('name', '').lower()
        if any(word in name for word in ['стажер', 'стажёр', 'интерн', 'помощник', 'intern', 'trainee']):
            grade = 'Intern'
        elif any(word in name for word in ['младший', 'junior', 'джуниор', 'начинающий']):
            grade = 'Junior'
        elif any(word in name for word in ['мидл', 'middle', 'средний', 'mid-level']):
            grade = 'Middle'
        elif any(word in name for word in ['сеньор', 'senior', 'старший', 'ведущий', 'опытный']):
            grade = 'Senior'
        elif any(word in name for word in ['тимлид', 'teamlead', 'руководитель', 'lead', 'главный']):
            grade = 'Team Lead'
        else:
            grade = 'Middle'


        row = {
            'vac_id': vac_annote.get('id'),
            'vac_name': vac_annote.get('name'),
            'grade':grade,
            'city': vac_annote.get('area', {}).get('name'),
            'geo': f"[{address.get('lat')}, {address.get('lng')}]" if address else None,
            'published_at': vac_annote.get('published_at'),
            'archived': vac_annote.get('archived'),
            'employer_id': employer.get('id') if employer else None,
            'emp_name': vac_annote.get('employment', {}).get('name'),
            'addres': address.get('raw') if address else None,
            'is_accredited': employer.get('accredited_it_employer') if employer else None,
            'is_trusted': employer.get('trusted') if employer else None,
            'salary_from': salary.get('from') if salary else None,
            'salary_to': salary.get('to') if salary else None,
            'currency': salary.get('currency') if salary else None,
            'gross': salary.get('gross') if salary else None,
            'mode_name': mode.get('name'),
            'frequency': mode.get('frequency'),
            'prof_role': prof_roles[0].get('name') if prof_roles else None,
            'schedule_name': vac_annote.get('schedule', {}).get('name'),
            'insider_interview': vac_annote.get('insider_interview'),
            'response_letter_required': vac_annote.get('response_letter_required'),
            'experience': vac_annote.get('experience', {}).get('name'),
            'key_skills': get_skills_str(vac_annote.get('key_skills')) if vac_annote.get('key_skills') else None,
            'has_test': vac_annote.get('has_test'),
            'description': vac_annote.get('description'),
            'url':vac_annote.get('alternate_url')
        }
        data_list.append(row)
        
    except Exception as e:
        print(f'Error processing vac_id {vac_id}: {e}')

# Создаем DataFrame одним вызовом
full_df = pd.DataFrame(data_list)
print('Success: Parsing done')

Получение описания вакансий...: 100%|██████████| 210/210 [01:57<00:00,  1.79it/s]

Success: Parsing done


In [62]:
geolocator = Nominatim(user_agent="geoapi", timeout=10)

def load_cache():
    # Загрузка кэша из файла
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}
    print('⚠️ Warning: Cache file not found')
    return {}


def get_geopoints(city: str, cache: dict) -> str:
    # Получение координат с использованием кэша
    if not city or pd.isna(city):
        return None
    
    # Проверяем кэш
    if city in cache:
        return cache[city]
    
    # Геокодирование если нет в кэше
    try:
        location = geolocator.geocode(f"{city}")
        if location:
            result = f'[{str(location.latitude)}, {str(location.longitude)}]'
        else:
            print(f'Error: Went wrong with {city}')
            result = None
    except Exception:
        result = None
    
    # Обновляем кэш
    cache[city] = result
    return result


def save_cache(cache):
    # Сохранение кэша в файл
    with open(CACHE_FILE, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

In [69]:
cache = load_cache()
mask = full_df.geo.isna()
full_df.loc[mask, 'geo'] = full_df.loc[mask, 'city'].apply(lambda x: get_geopoints(x, cache))

save_cache(cache)

--- 

In [ ]:
full_df.to_csv('resources/full_df.csv')
print('Success: DataFrame saved')

Success: DataFrame saved


In [20]:
path_to_gdisk = r'E:\Obsidian\YandexDisk'

full_df.to_excel(path_to_gdisk+'\\my.xlsx')



In [72]:
full_df

,vac_id,vac_name,grade,city,geo,published_at,archived,employer_id,emp_name,addres,is_accredited,is_trusted,salary_from,salary_to,currency,gross,mode_name,frequency,prof_role,schedule_name,insider_interview,response_letter_required,experience,key_skills,has_test,description,url
0,122617189,Младший аналитик данных,Junior,Санкт-Петербург,"[59.96466, 30.343303]",2025-07-10T10:53:50+0300,False,955804,Полная занятость,"Санкт-Петербург, Большой Сампсониевский проспе...",True,True,100000.0,120000.0,RUR,False,За месяц,None,"BI-аналитик, аналитик данных",Полный день,None,False,От 1 года до 3 лет,SQL; Работа с базами данных,False,<p><strong>Привет :)</strong></p> <p>Мы it ком...,https://hh.ru/vacancy/122617189
1,122540411,Аналитик данных,Middle,Санкт-Петербург,"[59.9606739, 30.1586551]",2025-07-08T16:42:47+0300,False,4334427,Полная занятость,None,True,True,200000.0,280000.0,RUR,False,За месяц,None,"BI-аналитик, аналитик данных",Полный день,None,False,От 3 до 6 лет,None,False,<p><strong>О проекте</strong></p> <p>Крупный к...,https://hh.ru/vacancy/122540411
2,122711825,Специалист по аналитике данных Яндекс,Middle,Краснодар,"[45.0351532, 38.9772396]",2025-07-11T14:23:51+0300,False,9498120,Полная занятость,None,False,True,38500.0,56550.0,RUR,True,За месяц,None,"BI-аналитик, аналитик данных",Удаленная работа,None,False,Нет опыта,Статистика; Статистическая отчетность; Статист...,False,<p><strong>Яндекс.Доставка </strong>- это серв...,https://hh.ru/vacancy/122711825
3,121091720,Intern Data Analyst / Стажер - аналитик данных,Intern,Москва,"[55.821573, 37.498725]",2025-07-11T10:24:56+0300,False,4949,Полная занятость,"Москва, Ленинградское шоссе, 16ас2",False,True,100000.0,NaN,RUR,True,За месяц,None,"BI-аналитик, аналитик данных",Полный день,None,False,Нет опыта,None,False,<p><strong>НАЙМ НА СРОЧНЫЙ ТРУДОВОЙ ДОГОВОР В ...,https://hh.ru/vacancy/121091720
4,122693922,Аналитик данных (отдел маркетинга),Middle,Санкт-Петербург,"[59.93442, 30.377627]",2025-07-11T09:33:10+0300,False,30151,Полная занятость,"Санкт-Петербург, 7-я Советская улица, 44",False,True,75000.0,NaN,RUR,False,За месяц,None,Маркетолог-аналитик,Полный день,None,False,Нет опыта,Аналитическое мышление; Анализ данных; MS Powe...,False,ЧТО ПРЕДСТОИТ ДЕЛАТЬ:​​​​​​​ <ul> <li> <p>Созд...,https://hh.ru/vacancy/122693922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,122701290,Аналитик базы данных недвижимости,Middle,Иваново (Ивановская область),"[55.730639, 37.62819]",2025-07-11T11:23:10+0300,False,2756,Полная занятость,"Москва, Валовая улица, 26",False,True,60000.0,120000.0,RUR,True,За месяц,None,"Оператор call-центра, специалист контактного ц...",Удаленная работа,None,False,Нет опыта,Работа с базами данных; Клиентоориентированнос...,False,<p>IBC Real Estate – лидирующая компания на ры...,https://hh.ru/vacancy/122701290
206,122701287,Аналитик базы данных недвижимости,Middle,Нижний Новгород,"[55.730639, 37.62819]",2025-07-11T11:23:10+0300,False,2756,Полная занятость,"Москва, Валовая улица, 26",False,True,60000.0,120000.0,RUR,True,За месяц,None,"Оператор call-центра, специалист контактного ц...",Удаленная работа,None,False,Нет опыта,Работа с базами данных; Клиентоориентированнос...,False,<p>IBC Real Estate – лидирующая компания на ры...,https://hh.ru/vacancy/122701287
207,122701288,Аналитик базы данных недвижимости,Middle,Смоленск,"[55.730639, 37.62819]",2025-07-11T11:23:10+0300,False,2756,Полная занятость,"Москва, Валовая улица, 26",False,True,60000.0,120000.0,RUR,True,За месяц,None,"Оператор call-центра, специалист контактного ц...",Удаленная работа,None,False,Нет опыта,Работа с базами данных; Клиентоориентированнос...,False,<p>IBC Real Estate – лидирующая компания на ры...,https://hh.ru/vacancy/122701288
208,122701284,Аналитик базы данных недвижимости,Middle,Тверь,"[55.730639, 37.62819]",2025-07-11T11:23:10+0300,False,2756,Полная занятость,"Москва, Валовая улица, 26",False,True,60000.0,120000.0,RUR,True,За месяц,None,"Опера